## Collection and Preparation:

In a MLOps approch, this jupyter notebook will Collect, Create the dataset and store it into a AWS like Bucket ( MiniIo).

Import and create the folder data:

In [2]:
! pip install pandas
! pip install tqdm
! pip install numpy
! pip install boto3
! mkdir data


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
mkdir: cannot create directory ‘data’: File exists


In [3]:
import os
import pandas as pd
import tqdm
import numpy as np
import random
import boto3
from botocore.exceptions import ClientError


In Montréal, in 2025, we have 15 skate spots. Some skatepark are greater to skate than others, for example, Ahuntsic 's skatepark have more modules than Verdun 's skate park.

Why create a model if you already know which parks is the best?
Depending on the traffic, on the day (and the occupation, because some skateparks are more busy the week end than others) the distance, we don't go to the same spot every session.

## The problem:
Ususally my friends ask me at the last minute: "Seb, where we go today?" And every time, I have no idea of which skatepark could be the best option. So that's a great problem to solve as a MLOps Software Engineer like me :) 


In [3]:
skateparks_dict = {
    "Ahuntsic": 8,
    "VanHorne": 4,
    "Verdun": 2,
    "Lasalle": 8.5,
    "Préfontaine": 8.5,
    "Boucherville": 7,
    "Taz": 7,
    "Spin": 6.5,
    "Saint Jérome": 8,
    "Saint Sauveur": 8.5,
    "Assomption": 8.5,
    "Benny": 8,
    "Dorval": 8,
    "Magog": 9.5,
    "Berthierville": 9.5
}


days_dict = {
    0: "Monday",
    1: "Tuesday",
    2: "Wenesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}

# Weather: 
# Rain or Snow  = 0
# Clouds + Wind = 1
# Clouds        = 2
# Blue sky      = 3

weather_dict = {
    0: "Rain or Snow",
    1: "Windy or heavily cloudy",
    2: "cloudy",
    3: "Blue sky"
}

Temperature will be in °C
traffic time: in minutes

We will give numbers index for the weekday:
0: Monday -> 6: Sunday

In [4]:
N=  20
data = []

for idx in tqdm.tqdm(range(N)):
    try:
        weather = random.randint(0, 3)
        temperature = np.clip(np.random.normal(22,6), -5, 38)
        traffic_time = np.clip(np.random.normal(25, 10), 5, 150)
        weekday = random.randint(0, 6)

        print(f"\n\nJour: {days_dict[weekday]}\t météo: {weather_dict[weather]}, a {temperature}°C\t, temps de traffic: {traffic_time}min")
        for idx, (spot_name, note) in enumerate(skateparks_dict.items()):
            print(f"{spot_name} (note: {note}/10)")
            satisfaction = input(f"Satisfaction?")
            data.append({
                "spot_name": spot_name,
                "index spot": idx,
                "weather": weather_dict[weather],
                "temperature": round(float(temperature), 1),
                "traffic_time": round(float(traffic_time), 1),
                "weekday": weekday,
                "park_score": note,
                "satisfaction": round(float(satisfaction), 2)
            })
    except Exception as e:
        print(e)


  0%|          | 0/20 [00:00<?, ?it/s]



Jour: Monday	 météo: cloudy, a 23.68254232817805°C	, temps de traffic: 10.062113275612838min
Ahuntsic (note: 8/10)


Satisfaction? 9


VanHorne (note: 4/10)


Satisfaction? 4


...

Taz (note: 7/10)


Satisfaction? 2


Spin (note: 6.5/10)


  0%|          | 0/20 [01:29<?, ?it/s]


KeyboardInterrupt: Interrupted by user

In [8]:

df = pd.DataFrame(data)
print(df.head(10))

df.to_csv("data/skatepark_dataset_test.csv", index=False)
print("\nDone")    

      spot_name  index spot  \
0      Ahuntsic           0   
1      VanHorne           1   
2        Verdun           2   
3       Lasalle           3   
4   Préfontaine           4   
5  Boucherville           5   
6           Taz           6   

                                             weather  temperature  \
0  {0: 'Rain or Snow', 1: 'Windy or heavily cloud...         23.7   
1  {0: 'Rain or Snow', 1: 'Windy or heavily cloud...         23.7   
2  {0: 'Rain or Snow', 1: 'Windy or heavily cloud...         23.7   
3  {0: 'Rain or Snow', 1: 'Windy or heavily cloud...         23.7   
4  {0: 'Rain or Snow', 1: 'Windy or heavily cloud...         23.7   
5  {0: 'Rain or Snow', 1: 'Windy or heavily cloud...         23.7   
6  {0: 'Rain or Snow', 1: 'Windy or heavily cloud...         23.7   

   traffic_time  weekday  park_score  satisfaction  
0          10.1        0         8.0           9.0  
1          10.1        0         4.0           4.0  
2          10.1        0         2.0   

## AWS like project:

We will use minIo, to use a local bucket like AWS S3. The goal is to be able to switch on AWS if in the future the project become more serious.
MinIO run In another Docker container, to have access to the GUI:

http://127.0.0.1:9001/login


In [4]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")

In [7]:
s3 = boto3.client(
    "s3",
    endpoint_url = S3_ENDPOINT,
    aws_access_key_id = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
    region_name="us-east-1"    
)

BUCKET_NAME = "skatedata"
FILE_PATH = "data/data.csv"
#FILE_PATH = "data/skatepark_dataset_test.csv"
OBJECT_NAME = "skatepark_dataset_test.csv"

try:
    s3.create_bucket(Bucket = BUCKET_NAME)
    print(f"[+] {BUCKET_NAME} bucket created!")
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print(f"[i] Bucket '{BUCKET_NAME}' already exists...")
    else:
        print(f"[-]{e}")

[i] Bucket 'skatedata' already exists...


In [8]:
s3.upload_file(FILE_PATH, BUCKET_NAME, OBJECT_NAME)

Check if the csv is inside the bucket

In [10]:
print("[i] file inside the bucket:")
for obj in s3.list_objects_v2(Bucket=BUCKET_NAME).get("Contents", []):
    print(f" - {obj['Key']}")

[i] file inside the bucket:
 - skatepark_dataset_test.csv
